# Mozilla TTS Training Example

Adapted [this gist](https://gist.github.com/erogol/97516ad65b44dbddb8cd694953187c5b) for [SoS notebook](https://vatlab.github.io/sos-docs/).

Assumes you have already [cloned TTS](https://github.com/mozilla/TTS) and done `apt install espeak`

## Prepare Training Data

In [ ]:
exit #comment to allow execution
wget http://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
tar -xjf LJSpeech-1.1.tar.bz2

In [ ]:
exit #comment to allow execution
shuf LJSpeech-1.1/metadata.csv > LJSpeech-1.1/metadata_shuf.csv
head -n 12000 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_train.csv
tail -n 1100 LJSpeech-1.1/metadata_shuf.csv > LJSpeech-1.1/metadata_val.csv

## Create config file for this training data

Some of my local paths are hardcoded below

In [ ]:
#exit #comment to allow execution
cd /vm/TTS
cp config.json "config.json.$(date +%Y%m%d)"
cat > config.json << EOM
{
    "model_name": "queue",
    "model_description": "Queue memory and change lower r incrementatlly",

    "audio":{
        // Audio processing parameters
        "num_mels": 80,         // size of the mel spec frame. 
        "num_freq": 1025,       // number of stft frequency levels. Size of the linear spectogram frame.
        "sample_rate": 22050,   // wav sample-rate. If different than the original data, it is resampled.
        "frame_length_ms": 50,  // stft window length in ms.
        "frame_shift_ms": 12.5, // stft window hop-lengh in ms.
        "preemphasis": 0.98,    // pre-emphasis to reduce spec noise and make it more structured. If 0.0, no -pre-emphasis.
        "min_level_db": -100,   // normalization range
        "ref_level_db": 20,     // reference level db, theoretically 20db is the sound of air.
        "power": 1.5,           // value to sharpen wav signals after GL algorithm.
        "griffin_lim_iters": 60,// #griffin-lim iterations. 30-60 is a good range. Larger the value, slower the generation.
        // Normalization parameters
        "signal_norm": true,    // normalize the spec values in range [0, 1]
        "symmetric_norm": false, // move normalization to range [-1, 1]
        "max_norm": 1,          // scale normalization to range [-max_norm, max_norm] or [0, max_norm]
        "clip_norm": true,      // clip normalized values into the range.
        "mel_fmin": null,         // minimum freq level for mel-spec. ~50 for male and ~95 for female voices. Tune for dataset!!
        "mel_fmax": null,        // maximum freq level for mel-spec. Tune for dataset!!
        "do_trim_silence": true  // enable trimming of slience of audio as you load it. LJspeech (false), TWEB (false), Nancy (true)
    },

    "distributed":{
        "backend": "nccl",
        "url": "tcp:\/\/localhost:54321"
    },

    "embedding_size": 256,  // Character embedding vector length. You don't need to change it in general.
    "text_cleaner": "phoneme_cleaners",
    "epochs": 1000,         // total number of epochs to train (originally 1000)
    "lr": 0.0001,            // Initial learning rate. If Noam decay is active, maximum learning rate.
    "lr_decay": false,      // if true, Noam learning rate decaying is applied through training.
    "loss_weight": 0.0,     // loss weight to emphasize lower frequencies. Lower frequencies are in general more important for speech signals.
    "warmup_steps": 4000,   // Noam decay steps to increase the learning rate from 0 to "lr"
    "windowing": false,      // Enables attention windowing. Used only in eval mode.
    "memory_size": 5,       // memory queue size used to queue network predictions to feed autoregressive connection. Useful if r < 5. 

    "batch_size": 32,       // Batch size for training. Lower values than 32 might cause hard to learn attention.
    "eval_batch_size":32,   
    "r": 5,                 // Number of frames to predict for step.
    "wd": 0.00001,          // Weight decay weight.
    "checkpoint": true,     // If true, it saves checkpoints per "save_step"
    "save_step": 5000,      // Number of training steps expected to save traning stats and checkpoints.
    "print_step": 50,       // Number of steps to log traning on console.
    "tb_model_param_stats": false,     // true, plots param stats per layer on tensorboard. Might be memory consuming, but good for debugging. 
    "batch_group_size": 8,  //Number of batches to shuffle after bucketing.

    "test_delay_epochs":1,
    "run_eval": true,
    "data_path": "/vm/LJSpeech-1.1/",  // can overwritten from command argument
    "meta_file_train": "metadata_train.csv",      // metafile for training dataloader
    "meta_file_val": "metadata_val.csv",    // metafile for validation dataloader
    "data_loader": "TTSDataset",      // dataloader, ["TTSDataset", "TTSDatasetCached", "TTSDatasetMemory"]
    "dataset": "ljspeech",     // one of TTS.dataset.preprocessors, only valid id dataloader == "TTSDataset", rest uses "tts_cache" by default.
    "min_seq_len": 0,       // DATASET-RELATED: minimum text length to use in training
    "max_seq_len": 300,     // DATASET-RELATED: maximum text length
    "output_path": "/vm/TTS/models/ljspeech_models/",
    "num_loader_workers": 2,
    "num_val_loader_workers": 2,
     "phoneme_cache_path": "ljspeech_phonemes",  // phoneme computation is slow, therefore, it caches results in the given folder.
    "use_phonemes": true,           // use phonemes instead of raw characters. It is suggested for better pronounciation.
    "phoneme_language": "en-us",     // depending on your target language, pick one from  https://github.com/bootphon/phonemizer#languages
    "text_cleaner": "phoneme_cleaners"
}
EOM

## Start Training

In [ ]:
cd /vm/TTS
python train.py --config_path config.json
#start tensorboard in another terminal
#tensorboard --logdir=my_run:<output_path in config.json>

## Continue Training

In [ ]:
python train.py --config_path config.json --restore_path /vm/TTS/models/ljspeech_models/queue-April-26-2019_03+46PM-59bbdb3
#/path/to/your/model.pth.tar

In [ ]:
import tensorflow as tf
hello = tf.constant("hello TensorFlow!")
sess=tf.Session() 
print(sess.run(hello))